# PySpark #01

Link da Aula: [PySpark - Aula 01 - Fundamentos](https://www.youtube.com/watch?v=ycZZs4371us)

**Indice:**

- [Importacão bibliotecas/funcões](#importacão-bibliotecasfuncões)
- [Iniciando sessão PySpark](#iniciando-sessão-pyspark)
- [Criando Data Frame](#criando-data-frame)
- [Verificando tipo das colunas](#verificando-tipo-das-colunas)
- [Renomeando Colunas](#renomeando-colunas)
- [Verificando a existencia de valores Nulos](#verificando-a-existencia-de-valores-nulos)
- [Selecionar Colunas com Select](#selecionar-colunas-com-select)
- [Selecionando Colunas com ALIAS](#selecionando-colunas-com-alias)
- [Organizar Select](#organizar-select)
- [Filtrar DF](#filtrar-df)
- [Filtrando DF com 2 condicoes ou mais (AND/&)](#filtrando-df-com-2-condicoes-ou-mais-and)
- [Filtrar DF com 2 condicoes (OR/|) ou mais](#filtrando-df-combinando--e--and-e-or)
- [Criar novas colunas (usando funcao lit)](#criar-novas-colunas-usando-funcao-lit)
- [Criar coluna condicional (usando funcao substring)](#criar-coluna-condicional-usando-funcao-substring)
- [Concatenar valores de colunas](#concatenando-valores-de-colunas)
- [Alterar tipos de coluna](#alterar-tipos-de-coluna)
- [Desafio: Coluna Nascimento](#desafio-coluna-nascimento)
 

##### Importacão bibliotecas/funcões

In [1]:
import os
import sys
#iniciando a sessao do pySpark
from pyspark.sql import SparkSession
#importando as funcoes
from pyspark.sql.functions import *
#importando os transformadores
from pyspark.sql.types import *

os.environ['PYSPARK_PYTHON'] = sys.executable
os.environ['PYSPARK_DRIVER_PYTHON'] = sys.executable


##### Iniciando sessão PySpark

* SparkSession.builder: Cria a sessao Spark
* .master('local'): Fala quem é o master, por que o Spark trabalha com Cluster. Entao aqui, é especificado quem é o master do cluster
* .appName('PySpark_01'): Nome da sua sessao, pode colocar o que quiser
* .getOrCreate(): Se caso vc já tenha criado uma sessao, ele vai puxar ela, se nao, ele cria uma nova.

In [2]:
spark = (
    SparkSession.builder
    .master('local')
    .appName('PySpark_01')
    .getOrCreate()
)

##### Criando Data Frame

>O PySpark tem a sintaxe parecida com a do Pandas, para importar um .csv, basta dar um 
```python:
spark.read.csv()
```

* Parametros:
    * header: informa se a primiera linha pode ser usada como nome das colunas.
    * inferSchema: Padroniza o typo dos dados das colunas.

In [3]:
df = spark.read.csv('wc2018-players.csv', header=True, inferSchema=True)

#Visualizando os 5 primeiros registros
df.show(5)

+---------+---+----+------------------+----------+----------+--------------------+------+------+
|     Team|  #|Pos.| FIFA Popular Name|Birth Date|Shirt Name|                Club|Height|Weight|
+---------+---+----+------------------+----------+----------+--------------------+------+------+
|Argentina|  3|  DF|TAGLIAFICO Nicolas|31.08.1992|TAGLIAFICO|      AFC Ajax (NED)|   169|    65|
|Argentina| 22|  MF|    PAVON Cristian|21.01.1996|     PAVÓN|CA Boca Juniors (...|   169|    65|
|Argentina| 15|  MF|    LANZINI Manuel|15.02.1993|   LANZINI|West Ham United F...|   167|    66|
|Argentina| 18|  DF|    SALVIO Eduardo|13.07.1990|    SALVIO|    SL Benfica (POR)|   167|    69|
|Argentina| 10|  FW|      MESSI Lionel|24.06.1987|     MESSI|  FC Barcelona (ESP)|   170|    72|
+---------+---+----+------------------+----------+----------+--------------------+------+------+
only showing top 5 rows



##### Verificando tipo das colunas

In [4]:
df.printSchema()

root
 |-- Team: string (nullable = true)
 |-- #: integer (nullable = true)
 |-- Pos.: string (nullable = true)
 |-- FIFA Popular Name: string (nullable = true)
 |-- Birth Date: string (nullable = true)
 |-- Shirt Name: string (nullable = true)
 |-- Club: string (nullable = true)
 |-- Height: integer (nullable = true)
 |-- Weight: integer (nullable = true)



##### Renomeando Colunas

" Diferente do Pandas, o Spark é imutável. Ou Seja, se voce subistituir o nome da coluna de um DF no Spark, nao dá pra usar o parametro inplace=True! Para resolver isso, voce deve fazer o código abaixo. "

```python:
df = df.withColumnRenamed('Nome Antigo','Novo Nome')
```

In [5]:
df = df.withColumnRenamed('Team','Selecao').withColumnRenamed('#','Numero')\
.withColumnRenamed('Pos.','Posicao').withColumnRenamed('FIFA Popular Name','Nome_FIFA')\
.withColumnRenamed('Birth Date','Nascimento')\
.withColumnRenamed('Shirt Name','Nome Camiseta').withColumnRenamed('Club','Time')\
.withColumnRenamed('Height','Altura').withColumnRenamed('Weight','Peso')

#Verificando colunas
df.columns

['Selecao',
 'Numero',
 'Posicao',
 'Nome_FIFA',
 'Nascimento',
 'Nome Camiseta',
 'Time',
 'Altura',
 'Peso']

##### Verificando a existencia de valores Nulos

" Diferente do Pandas, o Spark nao tem a funcao de idenfiticar os valores nulos. Para isso, precisamos aplicar o seguinte loop abaixo para contas valores vazios em cada coluna"

```python:

for coluna in df.columns:
    print(coluna, df.filter(df[coluna].isNull()).count())

In [6]:
for coluna in df.columns:
    print(coluna, df.filter(df[coluna].isNull()).count())


Selecao 0
Numero 0
Posicao 0
Nome_FIFA 0
Nascimento 0
Nome Camiseta 0
Time 0
Altura 0
Peso 0


##### Selecionar Colunas com Select

" A maioria das funcoes do PySpark sao baseadas no SQL, entao tem muitas funcoes parecidas."

In [7]:
df.select('selecao', 'nome_FIFA').show(5)

+---------+------------------+
|  selecao|         nome_FIFA|
+---------+------------------+
|Argentina|TAGLIAFICO Nicolas|
|Argentina|    PAVON Cristian|
|Argentina|    LANZINI Manuel|
|Argentina|    SALVIO Eduardo|
|Argentina|      MESSI Lionel|
+---------+------------------+
only showing top 5 rows



In [8]:
df.select(col('Selecao'), col('Altura')).show(5)

+---------+------+
|  Selecao|Altura|
+---------+------+
|Argentina|   169|
|Argentina|   169|
|Argentina|   167|
|Argentina|   167|
|Argentina|   170|
+---------+------+
only showing top 5 rows



In [9]:
df.select(df['Selecao']).show(5)

+---------+
|  Selecao|
+---------+
|Argentina|
|Argentina|
|Argentina|
|Argentina|
|Argentina|
+---------+
only showing top 5 rows



##### Selecionando Colunas com ALIAS

"Alias" é o apelido que voce dá para a coluna

In [10]:
df.select(col('Selecao').alias('Time')).show(5)

+---------+
|     Time|
+---------+
|Argentina|
|Argentina|
|Argentina|
|Argentina|
|Argentina|
+---------+
only showing top 5 rows



##### Organizar Select

"Chamar as colunas na ordem que voce quer"

In [11]:
#Ordem original das colunas
df.show(5)

+---------+------+-------+------------------+----------+-------------+--------------------+------+----+
|  Selecao|Numero|Posicao|         Nome_FIFA|Nascimento|Nome Camiseta|                Time|Altura|Peso|
+---------+------+-------+------------------+----------+-------------+--------------------+------+----+
|Argentina|     3|     DF|TAGLIAFICO Nicolas|31.08.1992|   TAGLIAFICO|      AFC Ajax (NED)|   169|  65|
|Argentina|    22|     MF|    PAVON Cristian|21.01.1996|        PAVÓN|CA Boca Juniors (...|   169|  65|
|Argentina|    15|     MF|    LANZINI Manuel|15.02.1993|      LANZINI|West Ham United F...|   167|  66|
|Argentina|    18|     DF|    SALVIO Eduardo|13.07.1990|       SALVIO|    SL Benfica (POR)|   167|  69|
|Argentina|    10|     FW|      MESSI Lionel|24.06.1987|        MESSI|  FC Barcelona (ESP)|   170|  72|
+---------+------+-------+------------------+----------+-------------+--------------------+------+----+
only showing top 5 rows



In [12]:
#Ordem personalizada
df.select('Nome_FIFA', 'Peso', 'Altura').show(5)

+------------------+----+------+
|         Nome_FIFA|Peso|Altura|
+------------------+----+------+
|TAGLIAFICO Nicolas|  65|   169|
|    PAVON Cristian|  65|   169|
|    LANZINI Manuel|  66|   167|
|    SALVIO Eduardo|  69|   167|
|      MESSI Lionel|  72|   170|
+------------------+----+------+
only showing top 5 rows



##### Filtrar DF

" Titulos de colunas que contem espaco no nome, retornam erro ao usar a fucao filtro, para resolver isso é legal colocar todos os nomes de colunas sem espaco ou usar a fucao col('Nome da Coluna')

In [13]:
#Metodo 1
df.filter('Selecao = "Brazil"').show(10)

+-------+------+-------+-----------------+----------+-------------+--------------------+------+----+
|Selecao|Numero|Posicao|        Nome_FIFA|Nascimento|Nome Camiseta|                Time|Altura|Peso|
+-------+------+-------+-----------------+----------+-------------+--------------------+------+----+
| Brazil|    18|     MF|             FRED|05.03.1993|         FRED|FC Shakhtar Donet...|   169|  64|
| Brazil|    21|     FW|           TAISON|13.01.1988|       TAISON|FC Shakhtar Donet...|   172|  64|
| Brazil|    17|     MF|      FERNANDINHO|04.05.1985|  FERNANDINHO|Manchester City F...|   179|  67|
| Brazil|    22|     DF|           FAGNER|11.06.1989|       FAGNER|SC Corinthians (BRA)|   168|  67|
| Brazil|    10|     FW|           NEYMAR|05.02.1992|    NEYMAR JR|Paris Saint-Germa...|   175|  68|
| Brazil|    11|     MF|PHILIPPE COUTINHO|12.06.1992|  P. COUTINHO|  FC Barcelona (ESP)|   172|  68|
| Brazil|     7|     FW|    DOUGLAS COSTA|14.09.1990|     D. COSTA|   Juventus FC (ITA)|   

In [14]:
#Metodo 2
df.filter(col("Nome Camiseta")=="FRED").show()

+-------+------+-------+---------+----------+-------------+--------------------+------+----+
|Selecao|Numero|Posicao|Nome_FIFA|Nascimento|Nome Camiseta|                Time|Altura|Peso|
+-------+------+-------+---------+----------+-------------+--------------------+------+----+
| Brazil|    18|     MF|     FRED|05.03.1993|         FRED|FC Shakhtar Donet...|   169|  64|
+-------+------+-------+---------+----------+-------------+--------------------+------+----+



##### Filtrando DF com 2 condicoes ou mais (AND/&)

"Cada condicao do filtro deve ser separada por parenteses"

In [15]:
df.filter((col('Selecao')== "Argentina") & (col('Altura')>180) & (col('Peso')>=85)).show()

+---------+------+-------+--------------+----------+-------------+--------------------+------+----+
|  Selecao|Numero|Posicao|     Nome_FIFA|Nascimento|Nome Camiseta|                Time|Altura|Peso|
+---------+------+-------+--------------+----------+-------------+--------------------+------+----+
|Argentina|     6|     DF|FAZIO Federico|17.03.1987|        FAZIO|       AS Roma (ITA)|   199|  85|
|Argentina|    12|     GK| ARMANI Franco|16.10.1986|       ARMANI|CA River Plate (ARG)|   189|  85|
|Argentina|     1|     GK| GUZMAN Nahuel|10.02.1986|       GUZMÁN|   Tigres UANL (MEX)|   192|  90|
+---------+------+-------+--------------+----------+-------------+--------------------+------+----+



##### Filtrar DF com 2 condicoes (OR/|) ou mais

In [16]:
df.filter((col('Nome_FIFA') == 'MESSI Lionel') | (col('Nome_FIFA')== 'SALVIO Eduardo') | (col('Altura') == 199)).show(5)

+---------+------+-------+----------------+----------+-------------+------------------+------+----+
|  Selecao|Numero|Posicao|       Nome_FIFA|Nascimento|Nome Camiseta|              Time|Altura|Peso|
+---------+------+-------+----------------+----------+-------------+------------------+------+----+
|Argentina|    18|     DF|  SALVIO Eduardo|13.07.1990|       SALVIO|  SL Benfica (POR)|   167|  69|
|Argentina|    10|     FW|    MESSI Lionel|24.06.1987|        MESSI|FC Barcelona (ESP)|   170|  72|
|Argentina|     6|     DF|  FAZIO Federico|17.03.1987|        FAZIO|     AS Roma (ITA)|   199|  85|
|  Belgium|     1|     GK|COURTOIS Thibaut|11.05.1992|     COURTOIS|  Chelsea FC (ENG)|   199|  91|
+---------+------+-------+----------------+----------+-------------+------------------+------+----+



##### Filtrando DF combinando & e | (AND e OR)

In [17]:
df.filter((col('Selecao')=='Brazil') & (col('Posicao')== 'DF') | (col('Altura') == '199') & (col('Selecao')=='Belgium')).show()

+-------+------+-------+----------------+----------+-------------+--------------------+------+----+
|Selecao|Numero|Posicao|       Nome_FIFA|Nascimento|Nome Camiseta|                Time|Altura|Peso|
+-------+------+-------+----------------+----------+-------------+--------------------+------+----+
|Belgium|     1|     GK|COURTOIS Thibaut|11.05.1992|     COURTOIS|    Chelsea FC (ENG)|   199|  91|
| Brazil|    22|     DF|          FAGNER|11.06.1989|       FAGNER|SC Corinthians (BRA)|   168|  67|
| Brazil|     6|     DF|     FILIPE LUIS|09.08.1985|  FILIPE LUIS|Atletico Madrid (...|   182|  73|
| Brazil|    13|     DF|      MARQUINHOS|14.05.1994|   MARQUINHOS|Paris Saint-Germa...|   183|  75|
| Brazil|     3|     DF|         MIRANDA|07.09.1984|      MIRANDA|FC Internazionale...|   186|  78|
| Brazil|    14|     DF|          DANILO|15.07.1991|       DANILO|Manchester City F...|   184|  78|
| Brazil|     2|     DF|    THIAGO SILVA|22.09.1984|     T. SILVA|Paris Saint-Germa...|   183|  79|


##### Criar novas colunas (usando funcao lit)

"A fucao lit cria uma coluna nova, com todos os valores iguais...

Caso voce queira colocar uma funcao nela, também é possível. Ex: Altura x Peso "

In [18]:
#Exemplo comum
df.withColumn('WorldCup', lit(2018)).show(5)

+---------+------+-------+------------------+----------+-------------+--------------------+------+----+--------+
|  Selecao|Numero|Posicao|         Nome_FIFA|Nascimento|Nome Camiseta|                Time|Altura|Peso|WorldCup|
+---------+------+-------+------------------+----------+-------------+--------------------+------+----+--------+
|Argentina|     3|     DF|TAGLIAFICO Nicolas|31.08.1992|   TAGLIAFICO|      AFC Ajax (NED)|   169|  65|    2018|
|Argentina|    22|     MF|    PAVON Cristian|21.01.1996|        PAVÓN|CA Boca Juniors (...|   169|  65|    2018|
|Argentina|    15|     MF|    LANZINI Manuel|15.02.1993|      LANZINI|West Ham United F...|   167|  66|    2018|
|Argentina|    18|     DF|    SALVIO Eduardo|13.07.1990|       SALVIO|    SL Benfica (POR)|   167|  69|    2018|
|Argentina|    10|     FW|      MESSI Lionel|24.06.1987|        MESSI|  FC Barcelona (ESP)|   170|  72|    2018|
+---------+------+-------+------------------+----------+-------------+--------------------+-----

In [19]:
#Exemplo com funcao
df.withColumn('Coluna_Nova', lit(col('Altura') * col('Peso'))).show(5)

+---------+------+-------+------------------+----------+-------------+--------------------+------+----+-----------+
|  Selecao|Numero|Posicao|         Nome_FIFA|Nascimento|Nome Camiseta|                Time|Altura|Peso|Coluna_Nova|
+---------+------+-------+------------------+----------+-------------+--------------------+------+----+-----------+
|Argentina|     3|     DF|TAGLIAFICO Nicolas|31.08.1992|   TAGLIAFICO|      AFC Ajax (NED)|   169|  65|      10985|
|Argentina|    22|     MF|    PAVON Cristian|21.01.1996|        PAVÓN|CA Boca Juniors (...|   169|  65|      10985|
|Argentina|    15|     MF|    LANZINI Manuel|15.02.1993|      LANZINI|West Ham United F...|   167|  66|      11022|
|Argentina|    18|     DF|    SALVIO Eduardo|13.07.1990|       SALVIO|    SL Benfica (POR)|   167|  69|      11523|
|Argentina|    10|     FW|      MESSI Lionel|24.06.1987|        MESSI|  FC Barcelona (ESP)|   170|  72|      12240|
+---------+------+-------+------------------+----------+-------------+--

##### Criar coluna condicional (usando funcao substring)

Com strings pode nao fazer muito sentido, mas com datas pode ser muito util!

In [25]:
df = df.withColumn('AnoNascimento', substring('Nascimento', -4, 4))
df.show(5)

+---------+------+-------+------------------+----------+-------------+--------------------+------+----+-------------+
|  Selecao|Numero|Posicao|         Nome_FIFA|Nascimento|Nome Camiseta|                Time|Altura|Peso|AnoNascimento|
+---------+------+-------+------------------+----------+-------------+--------------------+------+----+-------------+
|Argentina|     3|     DF|TAGLIAFICO Nicolas|31.08.1992|   TAGLIAFICO|      AFC Ajax (NED)|   169|  65|         1992|
|Argentina|    22|     MF|    PAVON Cristian|21.01.1996|        PAVÓN|CA Boca Juniors (...|   169|  65|         1996|
|Argentina|    15|     MF|    LANZINI Manuel|15.02.1993|      LANZINI|West Ham United F...|   167|  66|         1993|
|Argentina|    18|     DF|    SALVIO Eduardo|13.07.1990|       SALVIO|    SL Benfica (POR)|   167|  69|         1990|
|Argentina|    10|     FW|      MESSI Lionel|24.06.1987|        MESSI|  FC Barcelona (ESP)|   170|  72|         1987|
+---------+------+-------+------------------+----------+

##### Concatenar valores de colunas

Existem duas funcões para concater valores de colunas:
- **concat(col1, col2)**: Une valores de colunas sem separador
- **concat_ws(separador, col1, col2, colN)**: Une valores de colunas com reparador  

In [26]:
#Juntando colunas de forma grudada
df.withColumn('Concat', concat('Selecao', 'Nome Camiseta')).show(5)

+---------+------+-------+------------------+----------+-------------+--------------------+------+----+-------------+-------------------+
|  Selecao|Numero|Posicao|         Nome_FIFA|Nascimento|Nome Camiseta|                Time|Altura|Peso|AnoNascimento|             Concat|
+---------+------+-------+------------------+----------+-------------+--------------------+------+----+-------------+-------------------+
|Argentina|     3|     DF|TAGLIAFICO Nicolas|31.08.1992|   TAGLIAFICO|      AFC Ajax (NED)|   169|  65|         1992|ArgentinaTAGLIAFICO|
|Argentina|    22|     MF|    PAVON Cristian|21.01.1996|        PAVÓN|CA Boca Juniors (...|   169|  65|         1996|     ArgentinaPAVÓN|
|Argentina|    15|     MF|    LANZINI Manuel|15.02.1993|      LANZINI|West Ham United F...|   167|  66|         1993|   ArgentinaLANZINI|
|Argentina|    18|     DF|    SALVIO Eduardo|13.07.1990|       SALVIO|    SL Benfica (POR)|   167|  69|         1990|    ArgentinaSALVIO|
|Argentina|    10|     FW|      ME

In [27]:
#Juntando colunas com o separador
df.withColumn('Separador',concat_ws(' - ', 'Selecao', 'Numero', 'Posicao')).show(5)

+---------+------+-------+------------------+----------+-------------+--------------------+------+----+-------------+-------------------+
|  Selecao|Numero|Posicao|         Nome_FIFA|Nascimento|Nome Camiseta|                Time|Altura|Peso|AnoNascimento|          Separador|
+---------+------+-------+------------------+----------+-------------+--------------------+------+----+-------------+-------------------+
|Argentina|     3|     DF|TAGLIAFICO Nicolas|31.08.1992|   TAGLIAFICO|      AFC Ajax (NED)|   169|  65|         1992| Argentina - 3 - DF|
|Argentina|    22|     MF|    PAVON Cristian|21.01.1996|        PAVÓN|CA Boca Juniors (...|   169|  65|         1996|Argentina - 22 - MF|
|Argentina|    15|     MF|    LANZINI Manuel|15.02.1993|      LANZINI|West Ham United F...|   167|  66|         1993|Argentina - 15 - MF|
|Argentina|    18|     DF|    SALVIO Eduardo|13.07.1990|       SALVIO|    SL Benfica (POR)|   167|  69|         1990|Argentina - 18 - DF|
|Argentina|    10|     FW|      ME

##### Alterar tipos de coluna

>Para alterar o tipo da coluna, precisamos utilizar a funcão .cast()

Alguns Tipos:
- DateType()
- IntegerType()
- FloatType()

In [28]:
#Olhando o tipo de colunas
df.printSchema()

root
 |-- Selecao: string (nullable = true)
 |-- Numero: integer (nullable = true)
 |-- Posicao: string (nullable = true)
 |-- Nome_FIFA: string (nullable = true)
 |-- Nascimento: string (nullable = true)
 |-- Nome Camiseta: string (nullable = true)
 |-- Time: string (nullable = true)
 |-- Altura: integer (nullable = true)
 |-- Peso: integer (nullable = true)
 |-- AnoNascimento: string (nullable = true)



In [29]:
#Vamos alterar o tipo da coluna AnoNascimento para Inteiro
df.withColumn('AnoNascimento', col('AnoNascimento').cast(IntegerType())).show(5)

+---------+------+-------+------------------+----------+-------------+--------------------+------+----+-------------+
|  Selecao|Numero|Posicao|         Nome_FIFA|Nascimento|Nome Camiseta|                Time|Altura|Peso|AnoNascimento|
+---------+------+-------+------------------+----------+-------------+--------------------+------+----+-------------+
|Argentina|     3|     DF|TAGLIAFICO Nicolas|31.08.1992|   TAGLIAFICO|      AFC Ajax (NED)|   169|  65|         1992|
|Argentina|    22|     MF|    PAVON Cristian|21.01.1996|        PAVÓN|CA Boca Juniors (...|   169|  65|         1996|
|Argentina|    15|     MF|    LANZINI Manuel|15.02.1993|      LANZINI|West Ham United F...|   167|  66|         1993|
|Argentina|    18|     DF|    SALVIO Eduardo|13.07.1990|       SALVIO|    SL Benfica (POR)|   167|  69|         1990|
|Argentina|    10|     FW|      MESSI Lionel|24.06.1987|        MESSI|  FC Barcelona (ESP)|   170|  72|         1987|
+---------+------+-------+------------------+----------+

In [30]:
#Atribuindo alteracao no DF
df = df.withColumn('AnoNascimento', col('AnoNascimento').cast(IntegerType()))

#Verficando se a alteracao foi feita
df.printSchema()

root
 |-- Selecao: string (nullable = true)
 |-- Numero: integer (nullable = true)
 |-- Posicao: string (nullable = true)
 |-- Nome_FIFA: string (nullable = true)
 |-- Nascimento: string (nullable = true)
 |-- Nome Camiseta: string (nullable = true)
 |-- Time: string (nullable = true)
 |-- Altura: integer (nullable = true)
 |-- Peso: integer (nullable = true)
 |-- AnoNascimento: integer (nullable = true)



##### Desafio: Coluna Nascimento

- Colocar como DataType
- Dica: O formato precisa estar em: YYYY-MM-DD

In [31]:
#Visualizando df
df.show(5)

+---------+------+-------+------------------+----------+-------------+--------------------+------+----+-------------+
|  Selecao|Numero|Posicao|         Nome_FIFA|Nascimento|Nome Camiseta|                Time|Altura|Peso|AnoNascimento|
+---------+------+-------+------------------+----------+-------------+--------------------+------+----+-------------+
|Argentina|     3|     DF|TAGLIAFICO Nicolas|31.08.1992|   TAGLIAFICO|      AFC Ajax (NED)|   169|  65|         1992|
|Argentina|    22|     MF|    PAVON Cristian|21.01.1996|        PAVÓN|CA Boca Juniors (...|   169|  65|         1996|
|Argentina|    15|     MF|    LANZINI Manuel|15.02.1993|      LANZINI|West Ham United F...|   167|  66|         1993|
|Argentina|    18|     DF|    SALVIO Eduardo|13.07.1990|       SALVIO|    SL Benfica (POR)|   167|  69|         1990|
|Argentina|    10|     FW|      MESSI Lionel|24.06.1987|        MESSI|  FC Barcelona (ESP)|   170|  72|         1987|
+---------+------+-------+------------------+----------+

In [32]:
#Extraindo o dia
dia = udf(lambda data: data.split('.')[0])
df = df.withColumn('Dia', dia('Nascimento'))

#Extraindo o mes
mes = udf(lambda mes: mes.split('.')[1])
df = df.withColumn('Mes', mes('Nascimento'))

#Extraindo o ano
ano = udf(lambda ano: ano.split('.')[2])
df = df.withColumn('Ano', ano('Nascimento'))

#Visualizando colunas
df.printSchema()

root
 |-- Selecao: string (nullable = true)
 |-- Numero: integer (nullable = true)
 |-- Posicao: string (nullable = true)
 |-- Nome_FIFA: string (nullable = true)
 |-- Nascimento: string (nullable = true)
 |-- Nome Camiseta: string (nullable = true)
 |-- Time: string (nullable = true)
 |-- Altura: integer (nullable = true)
 |-- Peso: integer (nullable = true)
 |-- AnoNascimento: integer (nullable = true)
 |-- Dia: string (nullable = true)
 |-- Mes: string (nullable = true)
 |-- Ano: string (nullable = true)



##### Concatenando valores de colunas 

In [33]:
df = df.withColumn('Data_Nascimento', concat_ws('-', 'Ano', 'Mes', 'Dia'))
#Visualizando df
df.show(5)

+---------+------+-------+------------------+----------+-------------+--------------------+------+----+-------------+---+---+----+---------------+
|  Selecao|Numero|Posicao|         Nome_FIFA|Nascimento|Nome Camiseta|                Time|Altura|Peso|AnoNascimento|Dia|Mes| Ano|Data_Nascimento|
+---------+------+-------+------------------+----------+-------------+--------------------+------+----+-------------+---+---+----+---------------+
|Argentina|     3|     DF|TAGLIAFICO Nicolas|31.08.1992|   TAGLIAFICO|      AFC Ajax (NED)|   169|  65|         1992| 31| 08|1992|     1992-08-31|
|Argentina|    22|     MF|    PAVON Cristian|21.01.1996|        PAVÓN|CA Boca Juniors (...|   169|  65|         1996| 21| 01|1996|     1996-01-21|
|Argentina|    15|     MF|    LANZINI Manuel|15.02.1993|      LANZINI|West Ham United F...|   167|  66|         1993| 15| 02|1993|     1993-02-15|
|Argentina|    18|     DF|    SALVIO Eduardo|13.07.1990|       SALVIO|    SL Benfica (POR)|   167|  69|         1990| 

##### Convertendo tipo da coluna

In [35]:
df = df.withColumn('Data_Nascimento', col('Data_Nascimento').cast(DateType()))
df.show(5)

+---------+------+-------+------------------+----------+-------------+--------------------+------+----+-------------+---+---+----+---------------+
|  Selecao|Numero|Posicao|         Nome_FIFA|Nascimento|Nome Camiseta|                Time|Altura|Peso|AnoNascimento|Dia|Mes| Ano|Data_Nascimento|
+---------+------+-------+------------------+----------+-------------+--------------------+------+----+-------------+---+---+----+---------------+
|Argentina|     3|     DF|TAGLIAFICO Nicolas|31.08.1992|   TAGLIAFICO|      AFC Ajax (NED)|   169|  65|         1992| 31| 08|1992|     1992-08-31|
|Argentina|    22|     MF|    PAVON Cristian|21.01.1996|        PAVÓN|CA Boca Juniors (...|   169|  65|         1996| 21| 01|1996|     1996-01-21|
|Argentina|    15|     MF|    LANZINI Manuel|15.02.1993|      LANZINI|West Ham United F...|   167|  66|         1993| 15| 02|1993|     1993-02-15|
|Argentina|    18|     DF|    SALVIO Eduardo|13.07.1990|       SALVIO|    SL Benfica (POR)|   167|  69|         1990| 

In [36]:
#Visualizando tipo das colunas
df.printSchema()

root
 |-- Selecao: string (nullable = true)
 |-- Numero: integer (nullable = true)
 |-- Posicao: string (nullable = true)
 |-- Nome_FIFA: string (nullable = true)
 |-- Nascimento: string (nullable = true)
 |-- Nome Camiseta: string (nullable = true)
 |-- Time: string (nullable = true)
 |-- Altura: integer (nullable = true)
 |-- Peso: integer (nullable = true)
 |-- AnoNascimento: integer (nullable = true)
 |-- Dia: string (nullable = true)
 |-- Mes: string (nullable = true)
 |-- Ano: string (nullable = true)
 |-- Data_Nascimento: date (nullable = true)

